# Local code Q&A

This notebook provides an example of using a local model to ask questions about your code.


In [1]:
%pip install langchain chromadb

In [2]:
# load the document
from langchain.document_loaders import TextLoader

loader = TextLoader("../Program.cs")
documents = loader.load()

In [3]:
# split it into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter, Language

text_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.CSHARP)
all_splits = text_splitter.split_documents(documents)

In [4]:
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(
    documents=all_splits,
    embedding=OllamaEmbeddings(
        base_url="http://localhost:11434", model="codellama:13b-code-q4_K_M"
    ),  # type: ignore
)

In [ ]:
from langchain.chat_models import ChatOllama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
llm = ChatOllama(model="codellama:13b-code-q4_K_M", callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])) # type: ignore

%pip install langchainhub
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

Note: you may need to restart the kernel to use updated packages.


Ask your question here:

In [ ]:
# question = "How is the YouTube API being used?"
question = "The application currently isn't working correctly. Can you tell me what's wrong and how I can fix it?"

In [ ]:
result = qa_chain({"query": question})
result["result"]

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2
